# Task 2.4 - Concentric sampled deformation gradients

## Setup

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['text.usetex'] = True
sns.set_style('darkgrid')

from keras import optimizers
from keras import losses

from src.models import CustomFFNN, InvariantsICNN
from src.data_import import load_data
from src.plots import plot_stress_predictions, plot_energy_prediction, plot_loss, plot_stress_currelation
from src.analytic_potential import get_C_features
from src.predict_utils import predict_multi_cases_PANN, predict_multi_cases_naive
from src.utils import get_scores

2024-12-10 14:04:42.309808: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 14:04:42.313020: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 14:04:42.322496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 14:04:42.338645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 14:04:42.343621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 14:04:42.355348: I tensorflow/core/platform/cpu_feature_gu

### Paths

In [2]:
# Calibration paths
data_dir = os.path.abspath('task5_data')
biaxial_path = os.path.join(data_dir, 'BCC_biaxial.txt')
planar_path = os.path.join(data_dir, 'BCC_planar.txt')
shear_path = os.path.join(data_dir, 'BCC_shear.txt')
test1_path = os.path.join(data_dir, 'BCC_test1.txt')
test2_path = os.path.join(data_dir, 'BCC_test2.txt')
test3_path = os.path.join(data_dir, 'BCC_test3.txt')
uniaxial_path = os.path.join(data_dir, 'BCC_uniaxial.txt')
volumetric_path = os.path.join(data_dir, 'BCC_volumetric.txt')

In [ ]:
F_batch, P_batch, _ = load_data(biaxial_path, sep='\t')

ValueError: cannot reshape array of size 67 into shape (3,3)